In [1]:
import cv2
import numpy as np
import torch
from depth_anything_v2.dpt import DepthAnythingV2
import vtk
from PIL import Image
import math

xFormers not available
xFormers not available


In [4]:

model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vitb.pth', map_location='cuda:0')) #cpu


# model = DepthAnythingV2(encoder='vits', features=64, out_channels=[48, 96, 192, 384])
# model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vits.pth', map_location='cuda:0'))

def pic23d(pic_name,path,depth_hight = 10):
    model.cuda().eval()   #   cpu model.eval() 
    if ".png"in pic_name or ".webp" in pic_name:
        image = Image.open(pic_name)  # 或者 "example.png"

        # 转换并保存为 JPG 格式
        pic_name = pic_name.replace(".webp",".jpg").replace(".png",".jpg")
        image.convert("RGB").save(pic_name, "JPEG")
    raw_img = cv2.imread(path+pic_name)  
    height, width = raw_img.shape[:2]  
    target_height = 800  
    scale_ratio = target_height / height  
    target_width = int(width * scale_ratio)  
    raw_img = cv2.resize(raw_img, (target_width, target_height))  


    depth = model.infer_image(raw_img) # HxW raw depth map



    depth_normalized = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  
    depth_display = (depth_normalized * 255).astype(np.uint8)  # 转换为8位图像以便显示  
    cv2.imwrite('./depth_map.png', depth_display)  

    fx = 50  # 焦距x  
    fy = 50# 焦距y  
    cx = raw_img.shape[1] / 2.0  # 光心x  
    cy = raw_img.shape[0] / 2.0  # 光心y  
    height, width = raw_img.shape[:2]  
    
    # 推断深度图  
    depth_ = depth_display/255
    # 深度图可能需要归一化或转换到实际的深度值，这里假设它已经是所需的格式  
    
    # 创建一个空的数组来存储空间位置坐标  
    pos = np.zeros((height * width, 3), dtype=np.float32)  
    
    # 遍历深度图的每个像素  
    # Calculate indices for each pixel
    v_indices,u_indices = np.indices(depth_.shape)

    # Calculate spatial coordinates using vectorized operations
    X = (u_indices - cx) / fx
    Y = (v_indices - cy) / fy
    Z = depth_ * depth_hight

    # Stack the coordinates into a single array
    pos = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)
    
    # 现在pos数组包含了深度图上每个点的空间位置坐标
    # 计算面的面积
    def calculate_face_areas(pos, indices):
        areas = []
        for idx in indices:
            p0, p1, p2, p3 = pos[idx]
            # 计算两个三角形的面积
            v1 = p1 - p0
            v2 = p2 - p0
            v3 = p3 - p0
            area1 = np.linalg.norm(np.cross(v1, v2)) / 2.0
            area2 = np.linalg.norm(np.cross(v2, v3)) / 2.0
            areas.append(area1 + area2)
        return np.array(areas)
    all_face = False
    if  all_face == True:
        with open("a.obj","w") as t:
            t.writelines("mtllib my_mtl.mtl"+"\n")
                
            for l in pos:
                L = "v "+str(l[0])+" "+str(l[1])+" "+str(l[2])+"\n"
                t.writelines(L )
            
            w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
            h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
            t_array_ = np.stack([w_x,h_y],axis=1)


            for i,j in t_array_:
                L = "vt " +str(i)+" "+str(j) +"\n"
                t.writelines(L )                
        #  
        #          
            t.writelines("\nusemtl my_mtl\n" )              
            b = np.array(range(height * width)).reshape([ height ,width])
            print(b.shape)  


            for i in range(1,b.shape[0]-1):
                for j in range(1,b.shape[1]-1):
                    p = [ b[i,j] , b[i+1,j],b[i+1,j+1],b[i,j+1]]
            
                    L = "f "+str(p[0])+"/"+ str(p[0])+" "+str(p[1])+"/"+ str(p[1])+" "+str(p[2])+"/"+ str(p[2])+" "+str(p[3])+"/"+ str(p[3])+"\n"
                    t.writelines(L )


    else:

    #############################
        # 在生成面之后添加以下代码
        # 假设 b 是面索引的二维数组
        w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
        h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
        t_array_ = np.stack([w_x,h_y],axis=1)

        b = np.array(range(height * width)).reshape([ height ,width])
        face_indices = []
        for i in range(1, b.shape[0]-1):
            for j in range(1, b.shape[1]-1):
                p = [b[i,j], b[i+1,j], b[i+1,j+1], b[i,j+1]]
                face_indices.append(p)

        face_indices = np.array(face_indices)

        # 计算面面积
        areas = calculate_face_areas(pos, face_indices)

        # 删除面积大于0.02的面
        valid_indices = areas <= 0.015
        filtered_face_indices = face_indices[valid_indices]

        # 更新OBJ文件生成
        with open("filtered_model.obj", "w") as t:
            t.writelines("mtllib my_mtl.mtl" + "\n")

            for l in pos:
                L = "v " + str(l[0]) + " " + str(l[1]) + " " + str(l[2]) + "\n"
                t.writelines(L)

            # UV 坐标
            for i, j in t_array_:
                L = "vt " + str(i) + " " + str(j) + "\n"
                t.writelines(L)

            t.writelines("\nusemtl my_mtl\n")

            # 重新写入面
            for idx in filtered_face_indices:
                p = [idx[0], idx[1], idx[2], idx[3]]
                L = "f " + " ".join(f"{i + 1}/{i + 1}" for i in p) + "\n"  # 注意 OBJ 文件索引从1开始
                t.writelines(L)

#######################

        valid_indices = areas > 0.015
        filtered_face_indices = face_indices[valid_indices]

        # 更新OBJ文件生成
        with open("filtered_model_2.obj", "w") as t:
            t.writelines("mtllib green_mtl.mtl" + "\n")

            for l in pos:
                L = "v " + str(l[0]) + " " + str(l[1]) + " " + str(l[2]) + "\n"
                t.writelines(L)

            # UV 坐标
            for i, j in t_array_:
                L = "vt " + str(i) + " " + str(j) + "\n"
                t.writelines(L)

            t.writelines("\nusemtl green_mtl\n")

            # 重新写入面
            for idx in filtered_face_indices:
                p = [idx[0], idx[1], idx[2], idx[3]]
                L = "f " + " ".join(f"{i + 1}/{i + 1}" for i in p) + "\n"  # 注意 OBJ 文件索引从1开始
                t.writelines(L)
                
    with open("my_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl my_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd {pic_name}
    """
        )


    with open("green_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl green_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd green_texture.jpg
    """
        )



    def render_and_save_images(obj_files, texture_files, output_file, image_size=(800, 600)):
        # 读取模型

        # 创建渲染窗口
        render_window = vtk.vtkRenderWindow()       
        render_window.SetSize(image_size[0], image_size[1])  # 设置渲染窗口大小 
 



        # 创建左右视角的渲染器
        total_pic_count = 36
        renderers = []
        for i in range(total_pic_count):
            renderer = vtk.vtkRenderer()
            render_window.AddRenderer(renderer)
            renderers.append(renderer)
            # 禁用光照
            renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光







            for obj_file, pic_name in zip(obj_files, texture_files):
                # 读取模型
                reader = vtk.vtkOBJReader()
                reader.SetFileName(obj_file)
                reader.Update()
                
                # 创建一个映射器
                mapper = vtk.vtkPolyDataMapper()
                mapper.SetInputConnection(reader.GetOutputPort())
                
                # 创建一个演员
                actor = vtk.vtkActor()
                actor.SetMapper(mapper)
                
                # 读取贴图文件
                texture = vtk.vtkTexture()
                texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
                texture_reader.SetFileName(pic_name)  # 根据.mtl文件中的路径设置
                texture_reader.Update()
                texture.SetInputConnection(texture_reader.GetOutputPort())
                
                # 将贴图应用到演员
                actor.SetTexture(texture)
                renderer.AddActor(actor)


                actor.GetProperty().SetLighting(False)
            # 设置摄像机
            camera = vtk.vtkCamera()
            
            camera.SetFocalPoint(0, 0, 0)
            # if i == 0:  # 左视角
            #     camera.SetPosition(-2, 0, 30)  # 可以根据需要调整位置
            #     camera.SetViewUp(0, -1, 0)
            # else:  # 右视角
            #     camera.SetPosition(2, 0, 30)  # 右视角位置
            #     camera.SetViewUp(0, -1, 0)

      
            camera.SetPosition(-18+i*1, 0, 30)  # 可以根据需要调整位置
            camera.SetViewUp(0, -1, 0)

            renderer.SetActiveCamera(camera)

            # 添加演员到渲染器
            renderer.AddActor(actor)
            renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

            # 渲染
            render_window.Render()

            # 保存图像
            window_to_image_filter = vtk.vtkWindowToImageFilter()
            window_to_image_filter.SetInput(render_window)
            window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
            window_to_image_filter.Update()

            writer = vtk.vtkPNGWriter()
            writer.SetFileName(f'view_{i}.png')
            writer.SetInputConnection(window_to_image_filter.GetOutputPort())
            # writer.SetCompression(0)  # 设置为0表示无压缩，1表示有压缩（PNG默认是压缩）
            writer.Write()

        # 合并图片
        images = [Image.open(f'view_{i}.png') for i in [0,total_pic_count-1]]
        widths, heights = zip(*(i.size for i in images))

        total_width = sum(widths)
        max_height = max(heights)

        new_image = Image.new('RGB', (total_width, max_height))

        x_offset = 0
        for img in images:
            new_image.paste(img, (x_offset, 0))
            x_offset += img.width

        new_image.save(output_file)



        # 图片序列
        image_files = [f"view_{i}.png" for i in range(total_pic_count)]  # ["view_0.png", "view_1.png", ..., "view_9.png"]

        # 打开所有图片
        images = [Image.open(image_file) for image_file in image_files]

        # 将图片转换为 GIF
        images[0].save(
            'output.gif',
            save_all=True,
            append_images=images[1:],
            duration=100,  # 每帧持续时间，单位为毫秒
            loop=0  # 0 表示无限循环
    )
        
    obj_files = [ 'filtered_model_2.obj','filtered_model.obj']
    texture_files = ['green_texture.jpg','Q8.jpg', ]  # 贴图文件名

    render_and_save_images(obj_files, texture_files, 'output.png')

path = "./"
pic_name = "Q8.webp"
pic23d(pic_name,path,10)

In [8]:
# import vtk

# def render_and_save_images(obj_files, pic_names, output_prefix):
#     # 创建渲染窗口
#     render_window = vtk.vtkRenderWindow()
    
#     # 创建左右视角的渲染器
#     total_pic_count = 36
#     renderers = []
    
#     for i in range(total_pic_count):
#         renderer = vtk.vtkRenderer()
#         render_window.AddRenderer(renderer)
#         renderers.append(renderer)
        
#         # 禁用光照
#         renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光
        
#         for obj_file, pic_name in zip(obj_files, pic_names):
#             # 读取模型
#             reader = vtk.vtkOBJReader()
#             reader.SetFileName(obj_file)
#             reader.Update()
            
#             # 创建一个映射器
#             mapper = vtk.vtkPolyDataMapper()
#             mapper.SetInputConnection(reader.GetOutputPort())
            
#             # 创建一个演员
#             actor = vtk.vtkActor()
#             actor.SetMapper(mapper)
            
#             # 读取贴图文件
#             texture = vtk.vtkTexture()
#             texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
#             texture_reader.SetFileName(pic_name)  # 根据.mtl文件中的路径设置
#             texture_reader.Update()
#             texture.SetInputConnection(texture_reader.GetOutputPort())
            
#             # 将贴图应用到演员
#             actor.SetTexture(texture)
#             renderer.AddActor(actor)
        
#         # 设置摄像机
#         camera = vtk.vtkCamera()
#         camera.SetFocalPoint(0, 0, 0)
#         camera.SetPosition(-18 + i * 1, 0, 30)  # 可以根据需要调整位置
#         camera.SetViewUp(0, -1, 0)
#         renderer.SetActiveCamera(camera)

#         # 设置背景颜色
#         renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

#         # 渲染
#         render_window.Render()

#         # 保存图像
#         window_to_image_filter = vtk.vtkWindowToImageFilter()
#         window_to_image_filter.SetInput(render_window)
#         window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
#         window_to_image_filter.Update()

#         writer = vtk.vtkPNGWriter()
#         writer.SetFileName(f'{output_prefix}_view_{i}.png')
#         writer.SetInputConnection(window_to_image_filter.GetOutputPort())
#         writer.Write()

# # 使用示例
# obj_files = ['filtered_model.obj', 'filtered_model_2.obj']
# pic_names = ['Q8.jpg', 'green_texture.jpg']  # 对应的贴图文件
# render_and_save_images(obj_files, pic_names, 'output')